In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn.functional as F
from torch.autograd import Variable

from utils import plot_omniglot_pairs
from data_loader import get_train_valid_loader, get_test_loader

%load_ext autoreload
%autoreload 2

In [2]:
data_dir = './data/'

In [3]:
train_loader, valid_loader = get_train_valid_loader(data_dir, 32, False)
test_loader = get_test_loader(data_dir, 32)

In [4]:
print("Train: {:,}".format(len(train_loader.dataset)))
print("Valid: {:,}".format(len(valid_loader.dataset)))
print("Test: {:,}".format(len(test_loader.dataset)))

Train: 30,000
Valid: 6,760
Test: 16,000


In [ ]:
x, y = next(iter(train_loader))

# grab 2 pairs
x = x[0:2, :]
y = y[0:2]

x = x.numpy()
x = np.transpose(x, (0, 1, 3, 4, 2))

y = y.numpy()

print("x: {}".format(x.shape))
print("y: {}".format(y.shape))

In [ ]:
plot_omniglot_pairs(x, y, save=True, name='omniglot.png')